In [1]:
# imports
import os
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
import ollama



In [2]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [3]:
ed = Website("https://aws.amazon.com/what-is-aws/?nc1=f_cc&ams%23interactive-card-vertical%23pattern-data.filter=%257B%2522filters%2522%253A%255B%255D%257D")


In [4]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [5]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [6]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [7]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [9]:
def summarize(url):
    website = Website(url)
    response = ollama.chat(model=MODEL, messages=messages_for(website))
    return response['message']['content']


In [10]:
summarize("https://aws.amazon.com/what-is-aws/?nc1=f_cc&ams%23interactive-card-vertical%23pattern-data.filter=%257B%2522filters%2522%253A%255B%255D%257D")

'**Summary of What is AWS? - Cloud Computing with AWS**\n\nAWS (Amazon Web Services) is a comprehensive cloud computing platform offering over 200 fully featured services from data centers globally. It provides a flexible and secure environment for customers to lower costs, become more agile, and innovate faster.\n\n**Key Features:**\n\n* Most functionality and features compared to other cloud providers\n* Largest community of customers and partners worldwide\n* Deepest functionality within its services\n* Largest global infrastructure with 117 Availability Zones in 37 Geographic Regions\n* Fastest pace of innovation with new technologies such as serverless computing and machine learning\n\n**Security:**\n\n* Most secure cloud computing environment available today, backed by a deep set of cloud security tools\n* Compliance with over 300 security standards and certifications\n\n**Customer Stories:**\n\nAWS has recognized as the Magic Quadrant Leader for Strategic Cloud Platform Services

In [11]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [12]:
display_summary("https://edwarddonner.com")

**Summary of the Website**

The website is owned by Edward Donner, a CTO and co-founder of Nebula.io. The main focus of the website appears to be promoting his work with Large Language Models (LLMs) in the field of talent management.

### News and Announcements

* **Upcoming Courses**: "Connecting my courses – become an LLM expert and leader" (May 28, 2025)
* **2025 AI Executive Briefing** (April 21, 2025)
* **The Complete Agentic AI Engineering Course** (January 23, 2025)
* **LLM Workshop – Hands-on with Agents – resources** (No specific date mentioned)

### About the Creator

Edward Donner is a developer and enthusiast of LLMs. He also has experience as the founder and CEO of an AI startup called untapt, which was acquired in 2021.